In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
print(0)

In [ ]:
X = df_train.drop(['label'], axis=1)
X_train = X.values
Y = df_train['label']
Y_np = Y.values
y_train_3 = (Y_np == 3)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

some_digit = X_train[14]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
plt.axis("off")
plt.show()

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_3)

sgd_clf.predict([some_digit])

In [ ]:
#Calcul of the exact rate : more than 90% ! But there's only 1 / 10 to have a 3
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_3, cv=3, scoring="accuracy")

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_3, cv=3)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_3, y_train_pred)

In [ ]:
#Precision / recall
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_3, y_train_pred)#0.68497054911647348 68% des fois où il dit 5 : c'est vrai
recall_score(y_train_3, y_train_pred) #0.8285451620317168 83% de la totalité des 5 est prédite

#On peut combiner precision_score avec le recall_score en une métrique unique : F1
from sklearn.metrics import f1_score
f1_score(y_train_3, y_train_pred)


In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_3, cv=3,
                             method="decision_function")

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_3, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.axvline(x=70000,color='red')
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.xlim([-700000, 700000])
plt.show()

In [ ]:
y_train_pred_90 = (y_scores > 70000)
print(precision_score(y_train_3, y_train_pred_90))

print(recall_score(y_train_3, y_train_pred_90))

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_3, y_scores)
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_3, y_scores)

In [ ]:
from  sklearn.ensemble  import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_3, cv=3,
                                    method="predict_proba")



In [ ]:
y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_3,y_scores_forest)



In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
roc_auc_score(y_train_3, y_scores_forest)


In [ ]:
forest_clf.fit(X_train, Y)
forest_predict = pd.DataFrame(forest_clf.predict(df_test))



In [ ]:
submit = pd.read_csv('../input/sample_submission.csv')
submit['Label'] = pd.DataFrame(forest_clf.predict(df_test))
submit.to_csv('Random file.csv', sep=',', encoding = 'utf-8', index=False)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=4)


knn_clf.fit(X_train, Y)
submitK = pd.read_csv('../input/sample_submission.csv')
submitK['Label'] = pd.DataFrame(knn_clf.predict(df_test))
submitK.to_csv('KNN file.csv', sep=',', encoding = 'utf-8', index=False)
